In [1]:
# Setup the Jupyter version of Dash
from jupyter_dash import JupyterDash

# Configure the necessary Python module imports for dashboard components
import dash_leaflet as dl
from dash import dcc
from dash import html
import plotly.express as px
from dash import dash_table
from dash.dependencies import Input, Output, State
import base64

# Configure OS routines
import os

# Configure the plotting routines
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# Import AnimalShelter class from CRUD module
from animalshelter import AnimalShelter

###########################
# Data Manipulation / Model
###########################
# Update with your username and password
username = "aacuser"
password = "passw0rd1"
host = "nv-desktop-services.apporto.com"
port = "30131"
db = "AAC"
collection = "animals"
shelter = AnimalShelter(username, password, host, port, db, collection)

# Class read method must support return of list object and accept projection JSON input
# Sending the read method an empty document requests all documents be returned
df = pd.DataFrame.from_records(shelter.read({}))

# Remove the '_id' column if it exists
if '_id' in df.columns:
    df.drop(columns=['_id'], inplace=True)

#########################
# Dashboard Layout / View
#########################
app = JupyterDash(__name__)

# Load image
image_filename = 'Grazioso_Logo.png' 
encoded_image = None
if os.path.exists(image_filename):
    try:
        with open(image_filename, 'rb') as f:
            encoded_image = base64.b64encode(f.read()).decode()
    except Exception as e:
        print(f"Error reading image file: {e}")
else:
    print("Image file not found")

app.layout = html.Div([
    html.Center(html.B(html.H1('T. Hannon CS-340 Dashboard'))),
    html.Img(src='data:image/png;base64,{}'.format(encoded_image)) if encoded_image else html.P("Image not found"),
    html.Hr(),
    html.Div([
        html.Label('Filter by Rescue Type'),
        dcc.Dropdown(
            id='filter-type',
            options=[
                {'label': 'Water Rescue', 'value': 'Water Rescue'},
                {'label': 'Mountain or Wilderness Rescue', 'value': 'Mountain or Wilderness Rescue'},
                {'label': 'Disaster or Individual Tracking', 'value': 'Disaster or Individual Tracking'},
                {'label': 'Reset', 'value': 'Reset'},
            ],
            value='Reset'
        )
    ]),
    html.Hr(),
    dash_table.DataTable(
        id='datatable-id',
        columns=[{"name": i, "id": i, "deletable": False, "selectable": True} for i in df.columns],
        data=df.to_dict('records'),
        page_size=10,
        style_table={'overflowX': 'auto'},
        row_selectable='single',
        selected_rows=[0],
        filter_action='native',
        sort_action='native',
        style_data_conditional=[
            {'if': {'row_index': 'odd'}, 'backgroundColor': 'rgb(248, 248, 248)'}
        ],
        style_header={'backgroundColor': 'rgb(230, 230, 230)', 'fontWeight': 'bold'}
    ),
    html.Br(),
    html.Hr(),
    html.Div(className='row', style={'display': 'flex'}, children=[
        html.Div(id='graph-id', className='col s12 m6'),
        html.Div(id='map-id', className='col s12 m6')
    ])
])

@app.callback(
    Output('datatable-id', 'data'),
    [Input('filter-type', 'value')]
)
def update_dashboard(filter_type):
    if filter_type == 'Reset':
        query = None
    else:
        query = {}
        if filter_type != 'All':
            if filter_type == 'Water Rescue':
                query = {
                    "breed": {"$in": ["Labrador Retriever Mix", "Chesapeake Bay Retriever", "Newfoundland"]},
                    "sex_upon_outcome": "Intact Female",
                    "age_upon_outcome_in_weeks": {"$gte": 26, "$lte": 156}
                }
            elif filter_type == 'Mountain or Wilderness Rescue':
                query = {
                    "breed": {"$in": ["German Shepherd", "Alaskan Malamute", "Old English Sheepdog", "Siberian Husky", "Rottweiler"]},
                    "sex_upon_outcome": "Intact Male"
                }
            elif filter_type == 'Disaster or Individual Tracking':
                query = {
                    "breed": {"$in": ["Doberman Pinscher", "German Shepherd", "Golden Retriever", "Bloodhound", "Rottweiler"]},
                    "sex_upon_outcome": "Intact Male",
                    "age_upon_outcome_in_weeks": {"$gte": 26, "$lte": 156}
                }

    data = shelter.read(searchData=query)

    df = pd.DataFrame.from_records(data)
    if '_id' in df.columns:
        df.drop(columns=['_id'], inplace=True)

    return df.to_dict('records') if query else df.to_dict('records')

@app.callback(
    Output('graph-id', 'children'),
    [Input('datatable-id', 'derived_virtual_data')]
)
def update_graphs(viewData):
    if viewData is None:
        return []
    
    dff = pd.DataFrame.from_dict(viewData)
    print(dff.head())  # Debugging: Check the first few rows of the DataFrame
    if 'breed' not in dff.columns:
        print("Error: 'breed' column is missing in the DataFrame")
        return []
    
    fig = px.pie(dff, names='breed', title='Breed Distribution')
    return [dcc.Graph(figure=fig)]

@app.callback(
    Output('map-id', 'children'),
    [Input('datatable-id', 'derived_virtual_data'),
     Input('datatable-id', 'derived_virtual_selected_rows')]
)
def update_map(viewData, index):
    if viewData is None:
        return []
    
    dff = pd.DataFrame.from_dict(viewData)
    if len(index) == 0 or 'breed' not in dff.columns:
        return []

    row = index[0]
    return [
        dl.Map(style={'width': '1000px', 'height': '500px'}, center=[30.75, -97.48], zoom=10, children=[
            dl.TileLayer(id='base-layer-id'),
            dl.Marker(position=[dff.iloc[row, 13], dff.iloc[row, 14]], children=[
                dl.Tooltip(dff.iloc[row, 4]),
                dl.Popup([html.H1("Animal Name"), html.P(dff.iloc[row, 9])])
            ])
        ])
    ]


app.run_server(debug=True)


Dash app running on http://127.0.0.1:19584/
   rec_num age_upon_outcome animal_id animal_type                     breed  \
0        1          3 years   A746874         Cat    Domestic Shorthair Mix   
1        3          2 years   A716330         Dog   Chihuahua Shorthair Mix   
2       10         3 months   A664290         Cat    Domestic Shorthair Mix   
3        6          5 years   A696004         Dog  Cardigan Welsh Corgi Mix   
4       12           1 year   A664843         Dog              Pit Bull Mix   

         color date_of_birth             datetime            monthyear  \
0  Black/White    2014-04-10  2017-04-11 09:00:00  2017-04-11T09:00:00   
1  Brown/White    2013-11-18  2015-12-28 18:43:00  2015-12-28T18:43:00   
2       Tortie    2013-09-01  2013-12-08 14:58:00  2013-12-08T14:58:00   
3  Sable/White    2010-01-27  2015-01-28 10:39:00  2015-01-28T10:39:00   
4  Brown/White    2013-06-09  2014-08-18 17:24:00  2014-08-18T17:24:00   

       name outcome_subtype outcome_